In [1]:
import pandas as pd
import numpy as np
import gzip
from collections import defaultdict
import math
import scipy.optimize
from sklearn import svm
import string
import random
import string
from sklearn import linear_model
from implicit import bpr
import csv
import json
from random import sample 

In [2]:
def make_json(file):   
    data = {}
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            key = rows['id']
            data[key] = rows
    return data
def make_json_u(file):   
    data = {}
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            key = rows['u']
            data[key] = rows
    return data
def make_json_i(file):   
    data = []
    with open(file, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)         
        for rows in csvReader:
            data.append((rows['user_id'], rows['recipe_id'], rows))
    return data

In [3]:
all_interactions = make_json_i('data/Raw_interactions.csv')
data_train = make_json_i('data/interactions_train.csv')
data_val = make_json_i('data/interactions_validation.csv')
data_test = make_json_i('data/interactions_test.csv')
all_interactions[0]

('38094',
 '40893',
 {'user_id': '38094',
  'recipe_id': '40893',
  'date': '2003-02-17',
  'rating': '4',
  'review': 'Great with a salad. Cooked on top of stove for 15 minutes.Added a shake of cayenne and a pinch of salt.  Used low fat sour cream.  Thanks.'})

In [4]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)
allItems = []
allUsers = []

for d in all_interactions:
    reviewsPerUser[d[0]].append(d[1])
    reviewsPerItem[d[1]].append(d[0])
    allItems.append(d[1])
    allUsers.append(d[0])

In [5]:
newTest = []
for u,i,d in data_test:
    newTest.append((u, i, 1))
    unplayed = list(set(allItems[:1000]) - set(reviewsPerUser[u]))
    newTest.append((u, sample(unplayed, 1)[0], 0))

In [6]:
userID, itemID = {},{}
flippedID = {}
for user, recipe, d in all_interactions:
    if not user in userID: userID[user] = len(userID)
    if not recipe in itemID: 
        itemID[recipe] = len(itemID)
        flippedID[len(itemID)] = recipe
nUsers,nItems = len(userID),len(itemID)
nItems

231637

In [7]:
Xui = scipy.sparse.lil_matrix((nUsers, nItems))
for u,r,d in data_train:
    Xui[userID[u],itemID[r]] = 1
for u,r,d in data_val:
    Xui[userID[u],itemID[r]] = 1
for u,r,d in newTest:
    Xui[userID[u],itemID[r]] = 1
Xui_csr = scipy.sparse.csr_matrix(Xui)

In [8]:
model = bpr.BayesianPersonalizedRanking(factors = 5)
model.fit(Xui_csr)

  0%|          | 0/100 [00:00<?, ?it/s]

In [ ]:
predictions = []
for u, i, d in newTest:
    comp = model.recommend(userID[u], Xui_csr[userID[u]], N = 20000)
    if itemID[i] in comp[0]:
        predictions.append((u, i, 1))
    else:
        predictions.append((u, i, 0))

sum([x == y for x,y in zip(newTest, predictions)]) / len(newTest)

# Testing threshold values for the baseline model

In [ ]:
#baseline
thresholds = [2, 1.75, 1.5, 1.33, 1.25, 1.2]
accuracies = []
for thresh in thresholds:
    recipeCount = defaultdict(int)
    totalMade = 0

    for user,recipe,_ in data_train:
        recipeCount[recipe] += 1
        totalMade += 1

    mostPopular = [(recipeCount[x], x) for x in recipeCount]
    mostPopular.sort()
    mostPopular.reverse()

    return1 = set()
    count = 0

    '''
    for ic, i in mostPopular:
        if ic > 1:
            return1.add(i)
        else:
            break
    '''

    for ic, i in mostPopular:
        count += ic
        return1.add(i)
        if count > totalMade/thresh: break

    predictions = []
    for u,i,d in newTest:
        if i in return1:
            predictions.append((u, i, 1))
        else:
            predictions.append((u, i, 0))

    accuracies.append((thresh, sum([x == y for x,y in zip(newTest, predictions)]) / len(newTest)))

In [ ]:
accuracies